In [2]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd

# Party Manifests

gathered via Party Manifestos API

In [4]:
# https://manifesto-project.wzb.eu/down/data/2024a/codebooks/codebook_MPDataset_MPDS2024a.pdf
with open("../manifesto_apikey.txt") as f:
    apikey = f.read() # create API key: register at https://manifesto-project.wzb.eu/signup -> profile -> create api key
version = "MPDS2024a" # "Manifesto Project Dataset (version 2024a)"
versionA = "2024-1" # other versioning format
country = "41" # Germany
election_years = ['2017','2021'] # 19th and 20th Bundestag

In [5]:
# get party keys (and info)
all_parties_url = f"https://manifesto-project.wzb.eu/api/v1/get_parties?api_key={apikey}&key={version}"
response = requests.get(all_parties_url) 
if response.status_code == 200:    
    all_parties = response.json()     
else:    
    print(f"Request failed with status code {response.status_code}")   

In [6]:
all_parties_df = pd.DataFrame(all_parties[1:], columns=all_parties[0])
# select only current german parties
german_parties_df = all_parties_df[(all_parties_df['country'] == "41") & (all_parties_df['year_max'] == "2021")]
# create dictionary with party name and ID
german_party_ID = {}
for _, row in german_parties_df.iterrows():
    german_party_ID[int(row['party'])] = row['abbrev']
# drop SSW
del german_party_ID[41912]
german_party_ID

{41113: '90/Greens',
 41223: 'LINKE',
 41320: 'SPD',
 41420: 'FDP',
 41521: 'CDU/CSU',
 41953: 'AfD'}

### get manifestos for each party for each date

In [7]:
def party_manifetos(apikey,version,parties,years):
    # prepare api URL
    result = {}
    for key,value in parties.items():
        for year in years:
            code = f"{key}_{year}"
            combi = f"&keys[]={code}09"
            print(combi)
            call_manifesto_url = f"https://manifesto-project.wzb.eu/api/v1/texts_and_annotations?api_key={apikey}{combi}&version={version}"
            # call api
            manifesto = requests.get(call_manifesto_url) 
            if manifesto.status_code == 200:    
                manifesto_raw = manifesto.json()     
            else:    
                print(f"Request failed with status code {manifesto.status_code}")
            filtered_data = [item for item in manifesto_raw['items'][0]['items'] if item.get('cmp_code') != 'NA']
            manifest_text = " ".join(item.get('text', '').strip() for item in filtered_data)
            result[code] = {"partyID": key, "party": value, "year": year, "manifest_text": manifest_text, "manifest_tokens": filtered_data}
    return result

In [8]:
german_manifestos = party_manifetos(apikey,versionA,german_party_ID,election_years)

&keys[]=41113_201709
&keys[]=41113_202109
&keys[]=41223_201709
&keys[]=41223_202109
&keys[]=41320_201709
&keys[]=41320_202109
&keys[]=41420_201709
&keys[]=41420_202109
&keys[]=41521_201709
&keys[]=41521_202109
&keys[]=41953_201709
&keys[]=41953_202109


In [9]:
rows = []
for party_data in german_manifestos.values():
    rows.append({
        'partyID': party_data['partyID'],
        'party': party_data['party'],
        'year': party_data['year'],
        'manifest_text': party_data['manifest_text'],
        'manifest_tokens': party_data['manifest_tokens'],
    })

# Create DataFrame
df = pd.DataFrame(rows)

In [10]:
df.to_csv("../data/german_manifestos.csv",index=False)